In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from src.model import SIREN
import matplotlib.colors as mcolors
from src.discrete_operators_3d import *
from src.diff_operators import gradient, divergence

In [ ]:
def evaluate( model, samples, max_batch=64**2, device=torch.device(0) ):
    # samples = ( amount_samples, 3 )    
    head = 0
    amount_samples = samples.shape[0]

    evaluations = np.zeros( (amount_samples, 1))
    gradients = np.zeros( (amount_samples, 3))
    totalvariation = np.zeros( (amount_samples, 3))
    laplacians = np.zeros( (amount_samples, 1))

    while head < amount_samples:        
        if torch.is_tensor(samples):
            inputs_subset = samples[head:min(head + max_batch, amount_samples), :]
        else:

            inputs_subset = torch.from_numpy(samples[head:min(head + max_batch, amount_samples), :]).float()
            
        inputs_subset = inputs_subset.to(device).unsqueeze(0)

        x, y =  model(inputs_subset).values()
        subset_grad = gradient(y,x)
        gradients[head:min(head + max_batch, amount_samples)] = subset_grad.squeeze(0).detach().cpu().numpy()[..., :]
        totalvariation[head:min(head + max_batch, amount_samples)] = gradient(subset_grad.norm(dim=-1), x ).squeeze(0).detach().cpu().numpy()
        laplacians[head:min(head + max_batch, amount_samples)] = divergence( subset_grad, x ).squeeze(0).detach().cpu().numpy()[..., :]
        evaluations[head:min(head + max_batch, amount_samples)] = y.squeeze(0).detach().cpu()
        head += max_batch

    return evaluations, gradients, laplacians, totalvariation

model = SIREN(
        n_in_features= 3,
        n_out_features=1,
        hidden_layer_config=[512,512,512,512],
        w0=30,
        ww=None,
        activation= 'sine'
)
model.load_state_dict( torch.load('results/bunny/experiment/models/model_best.pth', weights_only=True))

device = torch.device(1)
model.to(device)

N = 64
xs, ys, zs = np.meshgrid( np.linspace(-1,1,N), np.linspace(-1,1,N), np.linspace(-1,1,N) )
X = np.concatenate( [ xs.flatten()[...,None], ys.flatten()[...,None], zs.flatten()[...,None] ] , axis=1)

fX, dfX, LfX, tv = evaluate(  model, X, device=device )

## Autograd:

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

midPlaneMask = X[:,2] == -1 + (N//2 ) * (2/(N-1))
midPlane = X[ midPlaneMask ]
clip_val = min( np.abs(np.min(fX[midPlaneMask])), np.abs(np.max(fX[midPlaneMask])) )

axs[0][0].scatter(midPlane[:,0], midPlane[:,1], c=fX[midPlaneMask], cmap='bwr', vmin=-clip_val, vmax= clip_val)
axs[0][1].scatter(midPlane[:,0], midPlane[:,1], c=np.sum( dfX ** 2, axis=1)[midPlaneMask], cmap='plasma')
axs[1][0].scatter(midPlane[:,0], midPlane[:,1], c=LfX[midPlaneMask], cmap='viridis')
axs[1][1].scatter(midPlane[:,0], midPlane[:,1], c=np.linalg.norm(tv, axis=1)[midPlaneMask], cmap='hot')

axs[0][0].set_title(r'$f(\vec{x})$')
axs[0][1].set_title(r'$\left \| \nabla f(\vec{x}) ) \right \|$')
axs[1][0].set_title(r'$\Delta f(\vec{x})$')
axs[1][1].set_title(r'$\left \| \nabla \left \| \nabla f(\vec{x}) ) \right \|  \right \|$')

plt.tight_layout()
plt.show()

## Discrete Neumann:

In [ ]:
Gx, Gy, Gz = gridDx(N), gridDy(N), gridDz(N)
discreteDfXx, discreteDfXy, discreteDfXz = Gx @ fX, Gy @ fX, Gz @ fX
gradNorm = discreteDfXx.flatten() ** 2 + discreteDfXy.flatten() ** 2 + discreteDfXz.flatten() ** 2

L = gridLaplacian( N )
discreteLfX = L @ fX

discreteTV = np.sqrt( (Gx @ gradNorm) ** 2 + (Gy @ gradNorm) ** 2 + (Gz @ gradNorm) ** 2 )

fig, axs = plt.subplots(1, 3, figsize=(15, 5))
clip_val = min( np.abs(np.min(fX)), np.abs(np.max(fX)) )

K = 1
midPlaneMaskNoBorders = (
    midPlaneMask * 
    (X[:,0] > -1 + K * (2/(N-1))) * 
    (X[:,0] < 1- K * (2/(N-1))) * 
    (X[:,1] > -1 + K * (2/(N-1))) * 
    (X[:,1] < 1 - K * (2/(N-1)))
)
midPlaneNoBorders = X[midPlaneMaskNoBorders]

axs[0].scatter(midPlaneNoBorders[:,0], midPlaneNoBorders[:,1], c=gradNorm[midPlaneMaskNoBorders], cmap='plasma')
axs[1].scatter(midPlaneNoBorders[:,0], midPlaneNoBorders[:,1], c=discreteLfX[midPlaneMaskNoBorders], cmap='viridis')
axs[2].scatter(midPlaneNoBorders[:,0], midPlaneNoBorders[:,1], c=discreteTV[midPlaneMaskNoBorders], cmap='hot')

axs[0].set_title(r'$\left \| \nabla f(\vec{x}) ) \right \|$')
axs[1].set_title(r'$\Delta f(\vec{x})$')
axs[2].set_title(r'$\left \| \nabla \left \| \nabla f(\vec{x}) ) \right \|  \right \|$')

plt.tight_layout()
plt.show()

Diferencias:

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

e1 = np.abs( np.sum( dfX ** 2, axis=1)[midPlaneMaskNoBorders] - gradNorm[midPlaneMaskNoBorders])
e2 = np.abs( LfX[midPlaneMaskNoBorders] - discreteLfX[midPlaneMaskNoBorders])
e3 = np.abs( np.linalg.norm(tv, axis=1)[midPlaneMaskNoBorders] - discreteTV[midPlaneMaskNoBorders])

axs[0].scatter(midPlaneNoBorders[:,0], midPlaneNoBorders[:,1], c=e1, cmap='Greys')
axs[1].scatter(midPlaneNoBorders[:,0], midPlaneNoBorders[:,1], c=e2, cmap='Greys')
axs[2].scatter(midPlaneNoBorders[:,0], midPlaneNoBorders[:,1], c=e3, cmap='Greys')

axs[0].text(0.5, -0.9, f"Error: {np.mean(e1):.2f}")
axs[1].text(0.5, -0.9, f"Error: {np.mean(e2):.2f}")
axs[2].text(0.5, -0.9, f"Error: {np.mean(e3):.2f}")

axs[0].set_title(r'$\left \| \nabla f(\vec{x}) ) \right \|$')
axs[1].set_title(r'$\Delta f(\vec{x})$')
axs[2].set_title(r'$\left \| \nabla \left \| \nabla f(\vec{x}) ) \right \|  \right \|$')

plt.tight_layout()
plt.show()

## Discrete second order

In [ ]:
Gx, Gy, Gz = gridDx(N), gridDy(N), gridDz(N)
Gxx, Gyy, Gzz = gridDxx(N), gridDyy(N), gridDzz(N)

L = gridLaplacian( N )
discreteLfX = L @ fX

discreteLfXSO = Gxx @ fX + Gyy @ fX + Gzz @ fX

discreteLfXFO = Gx @ Gx @ fX + Gy @ Gy @ fX + Gz @ Gz @ fX

fig, axs = plt.subplots(1, 4, figsize=(20, 5))

axs[0].scatter(midPlaneNoBorders[:,0], midPlaneNoBorders[:,1], c=LfX[midPlaneMaskNoBorders], cmap='plasma')
axs[1].scatter(midPlaneNoBorders[:,0], midPlaneNoBorders[:,1], c=discreteLfX[midPlaneMaskNoBorders], cmap='plasma')
axs[2].scatter(midPlaneNoBorders[:,0], midPlaneNoBorders[:,1], c=discreteLfXSO[midPlaneMaskNoBorders], cmap='plasma')
axs[3].scatter(midPlaneNoBorders[:,0], midPlaneNoBorders[:,1], c=discreteLfXFO[midPlaneMaskNoBorders], cmap='plasma')

axs[0].set_title(r'$\Delta f(\vec{x})$')
axs[1].set_title(r'$L f(\vec{x})$')
axs[2].set_title(r'$\partial_{xx}^2 f(\vec{x}) + \partial_{yy}^2 f(\vec{x}) + \partial_{zz}^2 f(\vec{x})$')
axs[3].set_title(r'$\partial_x \partial_x f(\vec{x}) + \partial_y \partial_y f(\vec{x}) + \partial_z \partial_z f(\vec{x})$')

plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

e1 = np.abs( LfX[midPlaneMaskNoBorders] - discreteLfX[midPlaneMaskNoBorders])
e2 = np.abs( LfX[midPlaneMaskNoBorders] - discreteLfXSO[midPlaneMaskNoBorders])
e3 = np.abs( LfX[midPlaneMaskNoBorders] - discreteLfXFO[midPlaneMaskNoBorders])

axs[0].scatter(midPlaneNoBorders[:,0], midPlaneNoBorders[:,1], c=e1, cmap='Greys')
axs[1].scatter(midPlaneNoBorders[:,0], midPlaneNoBorders[:,1], c=e2, cmap='Greys')
axs[2].scatter(midPlaneNoBorders[:,0], midPlaneNoBorders[:,1], c=e3, cmap='Greys')

axs[0].text(0.5, -0.9, f"Error: {np.mean(e1):.2f}")
axs[1].text(0.5, -0.9, f"Error: {np.mean(e2):.2f}")
axs[2].text(0.5, -0.9, f"Error: {np.mean(e3):.2f}")

axs[0].set_title(r'$L f(\vec{x})$')
axs[1].set_title(r'$\partial_{xx}^2 f(\vec{x}) + \partial_{yy}^2 f(\vec{x}) + \partial_{zz}^2 f(\vec{x})$')
axs[2].set_title(r'$\partial_x \partial_x f(\vec{x}) + \partial_y \partial_y f(\vec{x}) + \partial_z \partial_z f(\vec{x})$')

plt.tight_layout()
plt.show()

## Intrinsic operators

In [ ]:
def normalmap( X, nx, ny, axis ):
    norm = np.sqrt( nx ** 2 + ny ** 2 )
    norm_nx = - ( nx ) / (norm)
    norm_ny = - ( ny ) / (norm)
    
    angles = np.arctan2( -norm_nx, -norm_ny )
    
    norm = mcolors.Normalize(vmin=-np.pi, vmax=np.pi)
    angles_normalized = norm(angles)
    axis.scatter( X[:,0], X[:,1], c=angles_normalized, cmap='twilight' )

In [ ]:
PGx, PGy, PGz = gridIntrinsicGradient( N, dfX )
PGx_B, PGy_B, PGz_B = gridIntrinsicGradientBertalmio( N, fX )

fig, axs = plt.subplots(1, 2, figsize=(10, 5))

normalmap( midPlaneNoBorders, (PGx @ fX)[midPlaneMaskNoBorders], (PGy @ fX)[midPlaneMaskNoBorders], axs[0] )
normalmap( midPlaneNoBorders, (PGx_B @ fX)[midPlaneMaskNoBorders], (PGy_B @ fX)[midPlaneMaskNoBorders], axs[1] )

axs[0].set_title(r'$P_{\nabla \psi} \nabla$')
axs[1].set_title(r'$P_{\nabla \psi} \nabla \quad \text{Bertalmio}$')

plt.tight_layout()
plt.show()

In [ ]:
L = gridLaplaceBeltrami( N, dfX )
LB = gridLaplaceBeltramiBertalmio( N, fX )

fig, axs = plt.subplots(1, 2, figsize=(10, 5))

axs[0].scatter(midPlaneNoBorders[:,0], midPlaneNoBorders[:,1], c=(L @ fX)[midPlaneMaskNoBorders], cmap='plasma')
axs[1].scatter(midPlaneNoBorders[:,0], midPlaneNoBorders[:,1], c=(LB @ fX)[midPlaneMaskNoBorders], cmap='plasma')

axs[0].set_title(r'$\nabla \cdot P_{\nabla \psi} \nabla$')
axs[1].set_title(r'$\nabla \cdot P_{\nabla \psi} \nabla \quad \text{Bertalmio}$')

plt.tight_layout()
plt.show()